In [1]:
import os
from torch.utils.data import DataLoader
from torchvision.models.video import mvit_v2_s,MViT_V2_S_Weights
from custom_dataset import CustomVidDataset
from torchvideotransforms import video_transforms, volume_transforms
import time

In [2]:
# video_transform_list = [
# 			video_transforms.Resize((224, 224)),
#             volume_transforms.ClipToTensor()]
video_transform_list = [MViT_V2_S_Weights.KINETICS400_V1.transforms()]
transforms = video_transforms.Compose(video_transform_list)

In [3]:
# data_load = CustomImageDataset('img/label.csv','img/')
data_vid_load = CustomVidDataset('data/test_label.csv','data/test',transform=transforms)

In [4]:
data_loader = DataLoader(
    data_vid_load,
    batch_size=30,
    shuffle=True
)

In [5]:
import torch
from torch import nn
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using mps device


In [6]:
pretained_mvit = mvit_v2_s(weights="KINETICS400_V1")
pretained_mvit.head = nn.Sequential(
            nn.Linear(768,512),
            nn.ReLU(),
            nn.Linear(512, 12),
            nn.Sigmoid()
        )
print(pretained_mvit)

MViT(
  (conv_proj): Conv3d(3, 96, kernel_size=(3, 7, 7), stride=(2, 4, 4), padding=(1, 3, 3))
  (pos_encoding): PositionalEncoding()
  (blocks): ModuleList(
    (0): MultiscaleBlock(
      (norm1): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
      (norm2): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
      (attn): MultiscaleAttention(
        (qkv): Linear(in_features=96, out_features=288, bias=True)
        (project): Sequential(
          (0): Linear(in_features=96, out_features=96, bias=True)
        )
        (pool_q): Pool(
          (pool): Conv3d(96, 96, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), groups=96, bias=False)
          (norm_act): Sequential(
            (0): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
          )
        )
        (pool_k): Pool(
          (pool): Conv3d(96, 96, kernel_size=(3, 3, 3), stride=(1, 8, 8), padding=(1, 1, 1), groups=96, bias=False)
          (norm_act): Sequential(
            (0): LayerNorm((96,

In [8]:
pretained_mvit = pretained_mvit.to(device)

In [9]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(pretained_mvit.parameters(), lr=1e-3)

In [10]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [11]:
epochs = 1
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(data_loader, pretained_mvit, loss_fn, optimizer)
print("Done!")

Epoch 1
-------------------------------


RuntimeError: shape '[30, 96, 8, 56, 56]' is invalid for input of size 451584000